In [1]:
%load_ext autoreload
%autoreload 2

In [19]:
import numpy as np
import airo_models
from pydrake.common import temp_directory
from pydrake.geometry import StartMeshcat
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.analysis import Simulator
from pydrake.geometry import MeshcatVisualizer
from pydrake.planning import RobotDiagramBuilder, SceneGraphCollisionChecker
from tools.visualization import add_meshcat_triad
from tools.scenes import *
from tools.building import *
from typing import Optional, Tuple

from pydrake.geometry import Meshcat, MeshcatVisualizer, MeshcatVisualizerParams, Role
from pydrake.planning import RobotDiagram, RobotDiagramBuilder
from pydrake.systems.framework import Context
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

ROBOT = "ur5e"

In [20]:
#
# Setup initial robot diagram
#

robot_diagram_builder = RobotDiagramBuilder()
scene_graph = robot_diagram_builder.scene_graph()
builder = robot_diagram_builder.builder()

# Adding Meshcat must also be done before finalizing
meshcat = Meshcat()
meshcat.SetCameraPose([-2.0, 0, 1.0], [0, 0, 0])
MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)

# Add visualizer for proximity/collision geometry
collision_params = MeshcatVisualizerParams(role=Role.kProximity, prefix="collision", visible_by_default=False)
MeshcatVisualizer.AddToBuilder(builder, scene_graph.get_query_output_port(), meshcat, collision_params)

# Add robot to builder
arm_index = add_ur_and_table_to_builder(robot_diagram_builder, ROBOT)
tool_index = add_probe_tool_to_builder(robot_diagram_builder, arm_index)

# Finalize build
diagram, context = finish_build(robot_diagram_builder, meshcat)
plant = diagram.plant()
plant_context = plant.GetMyContextFromRoot(context)

collision_checker = SceneGraphCollisionChecker(
    model=diagram,
    robot_model_instances=[arm_index, tool_index],
    edge_step_size=0.125,  # Arbitrary value: we don't use the CheckEdgeCollisionFree
    env_collision_padding=0.005,
    self_collision_padding=0.005,
)


INFO:drake:Meshcat listening for connections at http://localhost:7001
Failed to load material file(s). Use default material.
material [ 'probeMaterial0' ] not found in .mtl



INFO:drake:Allocating contexts to support implicit context parallelism 12


In [29]:
#
# Setup initial pose and planner
#

start_joints = np.deg2rad([0.1, -90.1, -90.1, -90.1, 90.1, 0.1])
plant.SetPositions(plant_context, arm_index, start_joints)
diagram.ForcedPublish(context)

add_meshcat_triad(meshcat, "World", X_W_Triad=RigidTransform(p=[0, 0, 0], rpy=RollPitchYaw([0, 0, 0])))

# Planner setup
from ur_analytic_ik import ur3e, ur5e

ur_ik = ur3e if ROBOT == "ur3e" else ur5e

from tools.ompl.single_arm_planner import SingleArmOmplPlanner
from tools.visualization import publish_joint_path

tcp_transform = np.identity(4)
tcp_transform[2, 3] += 0.03

def inverse_kinematics_fn(tcp_pose):
    solutions_1x6 = ur_ik.inverse_kinematics_with_tcp(np.ascontiguousarray(tcp_pose), tcp_transform)
    solutions = [solution.squeeze() for solution in solutions_1x6]
    return solutions

def forward_kinematics_fn(joints):
    return ur_ik.forward_kinematics_with_tcp(*joints.squeeze(), tcp_transform)

planner = SingleArmOmplPlanner(collision_checker.CheckConfigCollisionFree, inverse_kinematics_fn, forward_kinematics_fn)

In [30]:
#
# Class used to interface with robot
# 

from tools.interfaces.drake_robot import DrakeRobot
            
robot = DrakeRobot(diagram, context, arm_index, meshcat, tcp_transform, ur_ik)
robot.zeroTFSensor()

add_meshcat_triad(meshcat, "TCP Frame 0", X_W_Triad=RigidTransform(robot.getTCPPose()[1]))

In [31]:
#
# Controller
#

from tools.controllers.calibration import CalibrationController
from tools.visualization import *

controller = CalibrationController(robot, planner)

In [32]:
# Define target pose
transform = robot.getTCPPose()[1]
print(transform)

target_transform = transform
target_transform[0:3, 3] -= np.array([0, 0, 0.1])

# random_joints = np.random.uniform(-2 * np.pi, 2 * np.pi, 6)
# target_transform = np.array(ur5e.forward_kinematics(*random_joints))

print(target_transform)
add_meshcat_triad(meshcat, "TCP Frame 1", X_W_Triad=RigidTransform(target_transform))

# Calculate ik
ik_solutions = ur3e.inverse_kinematics(np.ascontiguousarray(target_transform))

# Visualize solutions
publish_ik_solutions(ik_solutions, 3, meshcat, diagram, context, arm_index)


# Check if ok
print(len(ik_solutions))
for solution in ik_solutions:

  # Calc fk
  tcp = ur3e.forward_kinematics(*solution.squeeze())
  
  if not np.allclose(tcp, target_transform, atol=1e-2):
    print("IK solution is not correct")

[[-9.11718215e-06 -9.99986276e-01 -5.23899405e-03  4.92240936e-01]
 [-9.99998493e-01  2.12940480e-08  1.73618725e-03 -1.32214885e-01]
 [-1.73616331e-03  5.23900199e-03 -9.99984769e-01  4.56010093e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[-9.11718215e-06 -9.99986276e-01 -5.23899405e-03  4.92240936e-01]
 [-9.99998493e-01  2.12940480e-08  1.73618725e-03 -1.32214885e-01]
 [-1.73616331e-03  5.23900199e-03 -9.99984769e-01  3.56010093e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0


In [33]:
add_meshcat_triad(meshcat, "TCP Frame 0", X_W_Triad=RigidTransform(robot.getTCPPose()[1]))
joint_t, time_t, transform = controller.translate_probe([0.0, 0, -0.1])
add_meshcat_triad(meshcat, "TCP Frame 1", X_W_Triad=RigidTransform(transform))
robot.publish_trajectory(joint_t, time_t)

2024-02-29 15:47:46.570 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:158 - IK returned 8 solutions.
2024-02-29 15:47:46.572 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:169 - Found 8/8 solutions within joint bounds.
2024-02-29 15:47:46.574 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:184 - Found 8/8 solutions actually valid.


2024-02-29 15:47:46.581 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:191 - Found 8/8 valid solutions.


Debug:   RRTConnect: Planner range detected to be 6.156239
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.095102 seconds
Info:    SimpleSetup: Path simplification took 0.192056 seconds and changed from 3 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.039510 seconds
Info:    SimpleSetup: Path simplification took 0.137975 seconds and changed from 3 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.056475 seconds
Info:    SimpleSetup: Pa

2024-02-29 15:47:58.634 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:220 - Found 6 paths towards IK solutions:
2024-02-29 15:47:58.636 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:221 - Path lengths: [5.542, 5.898, 5.332, 4.77, 0.344, 5.552]
2024-02-29 15:47:58.638 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:222 - Path distances: [5.542, 5.898, 5.332, 4.77, 0.344, 5.552]
2024-02-29 15:47:58.640 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:237 - Length of chosen solution (= shortest path): 0.344


Info:    RRTConnect: Created 143 states (79 start + 64 goal)
Info:    ProblemDefinition: Adding approximate solution from planner RRTConnect
Info:    Solution found in 5.023090 seconds
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (3 start + 2 goal)
Info:    Solution found in 0.057295 seconds
Info:    SimpleSetup: Path simplification took 0.329926 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification took 0.000002 seconds and changed from 2 to 2 states


In [41]:
orig_transform = robot.getTCPPose()[1]
jt, tt = controller.move_to(orig_transform)
robot.publish_trajectory(jt, tt)

2024-02-29 15:55:57.079 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:158 - IK returned 8 solutions.
2024-02-29 15:55:57.081 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:169 - Found 8/8 solutions within joint bounds.
2024-02-29 15:55:57.083 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:184 - Found 8/8 solutions actually valid.
2024-02-29 15:55:57.089 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:191 - Found 8/8 valid solutions.


Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.070275 seconds
Info:    SimpleSetup: Path simplification took 0.249940 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.085696 seconds
Info:    SimpleSetup: Path simplification took 0.614967 seconds and changed from 3 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.058910 seconds
Info:    SimpleSetup: Path simplification took 0.168095 seconds and changed from 3 

2024-02-29 15:56:09.528 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:220 - Found 6 paths towards IK solutions:
2024-02-29 15:56:09.530 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:221 - Path lengths: [5.529, 5.742, 5.496, 4.694, 0.0, 5.422]
2024-02-29 15:56:09.531 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:222 - Path distances: [5.529, 5.742, 5.496, 4.694, 0.0, 5.422]
2024-02-29 15:56:09.532 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:237 - Length of chosen solution (= shortest path): 0.000
ERROR:drake:Toppra failed to find the maximum path acceleration at knot 22/128.


Info:    Solution found in 5.040941 seconds
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.065128 seconds
Info:    SimpleSetup: Path simplification took 0.408768 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification took 0.000002 seconds and changed from 2 to 2 states


AttributeError: 'NoneType' object has no attribute 'end_time'

In [43]:
controller.home_probe([0, 0, -1], 1, 0.2, robot)

2024-02-29 15:56:45.117 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:158 - IK returned 8 solutions.
2024-02-29 15:56:45.118 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:169 - Found 8/8 solutions within joint bounds.
2024-02-29 15:56:45.122 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:184 - Found 8/8 solutions actually valid.
2024-02-29 15:56:45.132 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:191 - Found 4/8 valid solutions.


Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.100528 seconds
Info:    SimpleSetup: Path simplification took 0.362121 seconds and changed from 3 to 2 states


2024-02-29 15:56:46.682 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:220 - Found 4 paths towards IK solutions:
2024-02-29 15:56:46.684 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:221 - Path lengths: [5.501, 5.773, 0.67, 5.672]
2024-02-29 15:56:46.685 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:222 - Path distances: [5.501, 5.773, 0.67, 5.672]
2024-02-29 15:56:46.686 | INFO     | tools.ompl.single_arm_planner:plan_to_tcp_pose:237 - Length of chosen solution (= shortest path): 0.670


Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 6 states (4 start + 2 goal)
Info:    Solution found in 0.109237 seconds
Info:    SimpleSetup: Path simplification took 0.444194 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification took 0.000002 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 5 states (2 start + 3 goal)
Info:    Solution found in 0.072978 seconds
Info:    SimpleSetup: Path simplification took 0.196969 seconds and changed from 4 to 2 states
Info:    SimpleSetup: Path simplification took 0.000001 seconds and changed from 2 to 2 states
Info:    RRTConnect: Starting planning with 1 states already in datastructure
Info:    RRTConnect: Created 4 states (2 start + 2 goal)
Info:    Solution found in 0.065